<a href="https://colab.research.google.com/github/mallibus/TF2---colab/blob/master/Colab_5_Building_a_Recurrent_Neural_Network_in_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Installing the dependencies and setting up a GPU environment

In [0]:
#!pip install tensorflow-gpu==2.0.0.alpha0

## Step 2: Importing the libraries

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [6]:
tf.__version__

'2.0.0-alpha0'

## Step 3: Data Preprocessing

### Setting up the dataset parameters

In [0]:
number_of_words = 20000
max_len = 100

### Loading the IMDB dataset

Cell added to allow imdb load - otherwise I got error  
`ValueError: Object arrays cannot be loaded when allow_pickle=False`

The fix comes from [here](https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa)



In [0]:

import numpy as np
old = np.load
np.load = lambda *a,**k: old(*a,**k,allow_pickle=True)

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

np.load = old
del(old)

In [15]:
# This does not work
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

ValueError: ignored

## Explore some reviews

To convert back from list of numbers to words with some sense, there is some work to do. [Here is where the code comes from](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset).

In [68]:
NUM_WORDS=number_of_words # only use top  words
INDEX_FROM=3   # word index offset

word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
#print(' '.join(id_to_word[id] for id in X_train[0] ))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be p

In [69]:
import textwrap
ylabels = ['Negative','Positive']
size = 3
sample = np.random.choice(range(len(X_train)),size)
for i in sample:
    print(f'Review #{i}:{ylabels[y_train[i]]}')
    s = ' '.join(id_to_word[id] for id in X_train[i])
    print('\n'.join(textwrap.wrap(s, width=120, replace_whitespace=False)))
    print()



Review #5769:Negative
<START> just <UNK> this i don't want to waste too much time on this as most of the posters here put it better than i
ever could but i did want to say a few things br br i didn't know which was funnier redgrave chasing tiny <UNK> and
tripping over her nurse close wailing that her precious boy whom she and the mr had decided was a drunken loser has been
turned into <UNK> that the tone deaf ann <UNK> with peggy lee or the horrid cgi of crypt keeper annie gazing at her
younger self br br i never bought danes as the younger redgrave i didn't buy richardson and collette as sisters either
if meryl streep's daughter wants to be an actress she better get mama to give her a few lessons i had zero idea why any
girl or buddy would make fools of themselves over vapid stud du <UNK> harris ann's daughters are as whiny and <UNK> as
she luc is a retarded slacker on crack and i didn't give a rot about any of them evening gives chick flicks a bad name

Review #17768:Positive
<START>

### Padding all sequences to be the same length 

In [0]:
# Chainging variable name to preserve orginal dataset
X_train_p = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [0]:
# Chainging variable name to preserve orginal dataset
X_test_p = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

## Step 4: Building a Recurrent Neural Network

### Defining the model

In [0]:
model = tf.keras.Sequential()

### Adding the embedding layer

In [0]:
model.add(tf.keras.layers.Embedding(input_dim=number_of_words, output_dim=128, input_shape=(X_train.shape[1],)))

### Adding the LSTM layer

- units: 128
- activation: tanh

In [0]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

W0510 22:26:40.657396 140526441928576 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fce6e733198>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


### Adding the output layer

- units: 1
- activation: sigmoid

In [0]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling the model

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [0]:
model.fit(X_train, y_train, epochs=3, batch_size=128)

Epoch 1/3
25000/25000 [==============================] - 4s 173us/sample - loss: 0.4640 - accuracy: 0.7792
Epoch 2/3
25000/25000 [==============================] - 2s 84us/sample - loss: 0.2884 - accuracy: 0.8842
Epoch 3/3
25000/25000 [==============================] - 2s 84us/sample - loss: 0.2291 - accuracy: 0.9129


### Evaluating the model

In [0]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

25000/25000 [==============================] - 3s 136us/sample - loss: 0.3403 - accuracy: 0.8528


In [0]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8527600169181824
